In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Optional
from datetime import datetime

DATA_DIR = Path("../data_ecf").resolve()
OUTPUT_DIR = Path("../output").resolve()
METEO_RAW_PATH = DATA_DIR / "meteo_raw.csv"
METEO_CLEAN_PATH = OUTPUT_DIR / "meteo_clean.csv"


### Partie 2 : Nettoyage avance avec Pandas (3-4h)

**Competence evaluee : C2.2 - Traiter des donnees structurees avec un langage de programmation**

#### Etape 2.1 : Nettoyage des donnees meteo
- Charger `meteo_raw.csv` avec Pandas

In [2]:
df_meteo_raw = pd.read_csv(METEO_RAW_PATH.as_posix(), quotechar="\"")

df_meteo_raw.info()
df_meteo_raw.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252612 entries, 0 to 252611
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   commune                  252612 non-null  object 
 1   timestamp                252612 non-null  object 
 2   temperature_c            251383 non-null  object 
 3   humidite_pct             252612 non-null  float64
 4   rayonnement_solaire_wm2  252612 non-null  float64
 5   vitesse_vent_kmh         252612 non-null  float64
 6   precipitation_mm         252612 non-null  float64
dtypes: float64(4), object(3)
memory usage: 13.5+ MB


,commune,timestamp,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm
0,Saint-Etienne,09/15/2024 15:00:00,17.1,143.3,244.9,14.3,0.0
1,Bordeaux,21/07/2023 15:00,19.6,50.6,414.9,3.2,0.0
2,Montpellier,2023-09-18 20:00:00,18.3,65.7,218.4,13.6,0.0
3,Le Havre,01/03/2024 22:00:00,3.7,94.9,6.8,18.6,11.6
4,Lille,29/10/2024 20:00,14.0,42.9,781.8,4.0,0.0
5,Bordeaux,22/12/2023 13:00,4.4,36.9,796.4,6.1,0.0
6,Marseille,09/15/2023 21:00:00,22.5,86.8,5.8,32.6,0.0
7,Toulouse,30/05/2023 00:00,8.3,66.3,26.4,31.4,7.2
8,Bordeaux,2024-10-05T09:00:00,11.5,69.8,71.4,34.4,0.0
9,Toulon,2024-09-28T21:00:00,19.2,79.0,13.1,31.8,0.0


- Standardiser les formats de dates

In [3]:
def parse_multi_format_timestamp(timestamp_str: str) -> Optional[datetime]:
    if timestamp_str is None:
        return None
    formats = [
        "%Y-%m-%d %H:%M:%S",
        "%d/%m/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%Y-%m-%dT%H:%M:%S",
    ]
    for fmt in formats:
        try:
            return datetime.strptime(timestamp_str, fmt)
        except ValueError:
            continue
    return None


df_meteo_ts = df_meteo_raw.copy()
df_meteo_ts["timestamp"] = df_meteo_ts["timestamp"].apply(lambda ts_str: parse_multi_format_timestamp(ts_str))

print("Nombre de NaN dans la colonne timestamp après traitement : ", df_meteo_ts["timestamp"].isna().sum())

Nombre de NaN dans la colonne timestamp après traitement :  0


- Convertir les colonnes numeriques en gerant les erreurs

Seule la colonne temperature_c n'est pas casté en float.

In [4]:
def clean_value(value_str: str) -> Optional[float]:
    if value_str is None:
        return None
    try:
        clean_str = value_str.replace(",", ".")
        return float(clean_str)
    except (ValueError, AttributeError):
        return None

df_meteo_numeric = df_meteo_ts.copy()
df_meteo_numeric["temperature_c"] = df_meteo_numeric["temperature_c"].apply(lambda num_str: clean_value(num_str))

print("Nombre de NaN dans la colonne temperature_c avant traitement : ", df_meteo_raw["temperature_c"].isna().sum())
print("Nombre de NaN dans la colonne temperature_c après traitement : ", df_meteo_numeric["temperature_c"].isna().sum())

Nombre de NaN dans la colonne temperature_c avant traitement :  1229
Nombre de NaN dans la colonne temperature_c après traitement :  1229


- Corriger les valeurs aberrantes :
  - Temperatures hors [-40, 50] -> NaN puis interpolation
  - Humidite hors [0, 100] -> clipping
  - Rayonnement solaire negatif -> 0

In [5]:
# Remplacer les températures hors [-40, 50] par NaN
df_meteo_numeric.loc[
    (df_meteo_numeric['temperature_c'] < -40) | (df_meteo_numeric['temperature_c'] > 50),
    'temperature_c'
] = np.nan

# Trier par commune et timestamp pour l'interpolation
df_meteo_numeric = df_meteo_numeric.sort_values(['commune', 'timestamp'])

# Interpolation lineaire pour temperature par commune
before_na = df_meteo_numeric["temperature_c"].isna().sum()
df_meteo_numeric["temperature_c"] = df_meteo_numeric.groupby('commune')["temperature_c"].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)
after_na = df_meteo_numeric["temperature_c"].isna().sum()
print(f"  temperature_c: {before_na} -> {after_na} NaN (interpoles: {before_na - after_na})")

  temperature_c: 3206 -> 0 NaN (interpoles: 3206)


In [17]:
# Clipping humidité hors [0, 100]
humidite_sup100 = df_meteo_numeric["humidite_pct"].loc[lambda x : x > 100]
humidite_inf0 = df_meteo_numeric["humidite_pct"].loc[lambda x : x < 0]
print(f"  Nombre de valeurs inférieures à 0 avant traitement : {humidite_inf0.count()}")
print(f"  Nombre de valeurs supérieures à 100 avant traitement : {humidite_sup100.count()}")

df_meteo_numeric["humidite_pct"] = df_meteo_numeric["humidite_pct"].clip(0, 100)

  Nombre de valeurs inférieures à 0 avant traitement : 0
  Nombre de valeurs supérieures à 100 avant traitement : 0


In [18]:
# Rayonnement solaire négatif mis à 0
rayonnement_inf0 = df_meteo_numeric["rayonnement_solaire_wm2"].loc[lambda x : x < 0]
print(f"  Nombre de valeurs inférieures à 0 avant traitement : {rayonnement_inf0.count()}")

df_meteo_numeric.loc[
    df_meteo_numeric['rayonnement_solaire_wm2'] < 0,
    'rayonnement_solaire_wm2'
] = 0

  Nombre de valeurs inférieures à 0 avant traitement : 0


- Traiter les valeurs manquantes :
  - Interpolation lineaire pour temperature et humidite
  - Forward fill pour precipitation

In [8]:
# Interpolation linéaire pour humidité
before_na = df_meteo_numeric["humidite_pct"].isna().sum()
df_meteo_numeric["humidite_pct"] = df_meteo_numeric.groupby('commune')["humidite_pct"].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)
after_na = df_meteo_numeric["humidite_pct"].isna().sum()
print(f"  humidite_pct: {before_na} -> {after_na} NaN (interpoles: {before_na - after_na})")

  humidite_pct: 0 -> 0 NaN (interpoles: 0)


In [9]:
# Forward fill pour précipitation
df_meteo_numeric["precipitation_mm"] = df_meteo_numeric.groupby('commune')["precipitation_mm"].transform(lambda x: x.ffill().bfill())

- Ajouter des colonnes temporelles (jour, mois, saison, jour de semaine)

In [10]:
df_meteo_numeric['date'] = df_meteo_numeric['timestamp'].dt.date
df_meteo_numeric['day_of_week'] = df_meteo_numeric['timestamp'].dt.dayofweek
df_meteo_numeric['month'] = df_meteo_numeric['timestamp'].dt.month
df_meteo_numeric['season'] = df_meteo_numeric['month'].map({
    12: 'Hiver', 1: 'Hiver', 2: 'Hiver',
    3: 'Printemps', 4: 'Printemps', 5: 'Printemps',
    6: 'Ete', 7: 'Ete', 8: 'Ete',
    9: 'Automne', 10: 'Automne', 11: 'Automne'
})

In [11]:
# Sauvegarde csv
df_meteo_numeric.to_csv(METEO_CLEAN_PATH, index=False)

In [12]:
# Rapport final
print("RAPPORT DE NETTOYAGE - DONNEES METEO")
print(f"Lignes brutes: {len(df_meteo_raw):,}")
print(f"Lignes apres nettoyage: {len(df_meteo_numeric):,}")
print(f"Lignes supprimees: {len(df_meteo_raw) - len(df_meteo_numeric):,}")
print("\nCompletude finale:")
for col in ['temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2',	'vitesse_vent_kmh', 'precipitation_mm']:
    completude = (1 - df_meteo_numeric[col].isna().sum() / len(df_meteo_raw)) * 100
    print(f"  {col}: {completude:.2f}%")

RAPPORT DE NETTOYAGE - DONNEES METEO
Lignes brutes: 252,612
Lignes apres nettoyage: 252,612
Lignes supprimees: 0

Completude finale:
  temperature_c: 100.00%
  humidite_pct: 100.00%
  rayonnement_solaire_wm2: 100.00%
  vitesse_vent_kmh: 100.00%
  precipitation_mm: 100.00%
